In [37]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Model
import torch.nn as nn

# Load pre-trained model and tokenizer
model_name = "gpt2-medium"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Set the model to evaluation mode
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [46]:
t: GPT2Model = model.transformer
def hook(module, args, output):
    print(f"Input shape: {args[0][0,-1,:].shape}")
    print(f"Output shape: {output[0][0,-1,:].shape}")

hooks = []
for layer in t.h[:1]:
    h = layer.register_forward_hook(hook)
    hooks.append(h)
    
input = tokenizer.encode("My wife works as a", return_tensors="pt")
# Run the model to get outputs and capture intermediate representations
with torch.no_grad():
    outputs = model(input)

# Extract the generated text
logits = outputs.logits
generated_ids = torch.argmax(logits, dim=-1)
generated_text = tokenizer.decode(generated_ids[0][-1])

# Print the generated text
print(f"Generated next token: {generated_text}")

# Remove the hooks
for h in hooks:
    h.remove()

Input shape: torch.Size([1024])
Output shape: torch.Size([1024])
Generated next token:  nurse


In [ ]:
model.trans

In [73]:
def get_intermediate_representation(module, input, output):
    print(f"Intermediate representation: {input}")
    
# Register the hook to the first layer of the model
model.transformer.h[0].register_forward_hook(get_intermediate_representation)


# Prepare a dummy input for inference
input_text = "My wife works as a"
input_ids = tokenizer.encode(input_text, return_tensors="pt")
model(input_ids)

AttributeError: 'GPT2Block' object has no attribute 'unregister_forward_hook'

In [61]:
# Define a hook to capture intermediate representations
def get_intermediate_representation(module, input, output):
    layer_outputs.append(output)

# Prepare a dummy input for inference
input_text = "The quick brown fox jumps over the lazy dog."
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# List to store intermediate representations and FFN weights
layer_outputs = []
weights = []

# Register hooks for each layer
for i, block in enumerate(model.transformer.h):
    block.mlp.register_forward_hook(get_intermediate_representation)

# Run the model to get outputs and capture intermediate representations
with torch.no_grad():
    outputs = model(input_ids)

# Extract the generated text
logits = outputs.logits
generated_ids = torch.argmax(logits, dim=-1)
generated_text = tokenizer.decode(generated_ids[0])

# Print the generated text
print(f"Generated text: {generated_text}")

# Output the intermediate representations and weights of each FFN layer
print(f"Intermediate representations (before and after each FFN):")
for i, output in enumerate(layer_outputs):
    print(f"Layer {i} - Shape: {output.shape}")

print(f"\nWeights (up and down for each FFN):")
for i, weight in enumerate(weights):
    print(f"Layer {i} - Shape: {weight.shape}")


AttributeError: 'Conv1D' object has no attribute 'attn'